# Plate Modal Analysis
As an exploration to learn how to us e the fencis program this will be as imple plate problem to resolvet he solutions.

Code largely taken from amessage board disucssing this same problem
https://fenicsproject.discourse.group/t/modal-analysis-using-dolfin-x/7349/4

## Expand MPI in jupyter
- Install ipyparallel
- Enable it in Jupyter
- jupyter serverextension enable --py ipyparallel
- ipcluster start --n=4

In [2]:

from contextlib import ExitStack

import numpy as np

from dolfinx import la
from dolfinx.mesh import CellType, create_box, locate_entities_boundary

from dolfinx.fem import (DirichletBC, Function, VectorFunctionSpace,
                         apply_lifting, assemble_matrix, assemble_vector,
                         locate_dofs_geometrical, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, GhostMode, create_box
from ufl import (Identity, SpatialCoordinate, TestFunction, TrialFunction,
                 as_vector, dx, grad, inner, sym, tr,CellVolume)
from dolfinx.fem import (Constant, DirichletBC, Function, LinearProblem, FunctionSpace, VectorFunctionSpace, 
                         locate_dofs_topological)
from petsc4py.PETSc import ScalarType
from dolfinx import fem
import dolfinx
import matplotlib.pyplot as plt
import ufl
from mpi4py import MPI
from slepc4py import SLEPc
# from ipyparallel import Client, error
# cluster = Client(profile="mpi")

In [3]:
# invoke mpi
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f"size is {size}")
print(f"rank is {rank}")

size is 1
rank is 0


In [4]:
#create the mesh

# define the dimensions
x_0 = np.array([0.0,0.0,0.0])
x_1 = np.array([2.0,1.0,1.0])

# n_elements
elements = [100,100,15]

# mesh = create_box(
    # comm, [x_0,x_1],elements,
    # CellType.tetrahedron, GhostMode.shared_facet)

L,B,H = 20.,0.5,1.
Nx = 200
Ny = int(B/L*Nx)+1
Nz = int(H/L*Nx)+1
mesh = create_box(
    comm, [np.array([0.0, 0.0, 0.0]),
                     np.array([L,B,H])], [Nx,Ny,Nz],
    CellType.tetrahedron, GhostMode.shared_facet)


In [5]:
num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
num_cells


79200

In [6]:
# define the constants, using Aluminum
E = 70e9
nu = 0.33
rho = 2700

# define the lame constant
mu = E/2./(1.+nu)
lmbda = E*nu/(1.+nu)/(1.-2.*nu)
lambda_ = E*nu/(1+nu)/(1-2*nu)


In [7]:
#define some functions
def epsilon(v):
    return ufl.sym(ufl.grad(v))

def sigma(v):
    dim=v.geometric_dimension()
    return 2.0*mu*epsilon(v)+lmbda*ufl.tr(epsilon(v))*ufl.Identity(dim)

In [8]:
# def epsilon(u):
#     return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
# def sigma(u):
#     return lambda_ * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

In [9]:
# build the FEM solution
V = VectorFunctionSpace(mesh,('Lagrange',1))
u = TrialFunction(V)
v = TestFunction(V)



In [10]:
# Define the boundary conditions
def left_clamp(x):
    return np.isclose(x[0], 0)# | (np.isclose(x[0],1.0)) | (np.isclose(x[1],1.0)) | (np.isclose(x[1],0.0))

fdim = mesh.topology.dim - 1
boundary_facets = locate_entities_boundary(mesh, fdim, left_clamp)

u_D = Function(V)
u_D.x.array[:] = 0
bc = DirichletBC(u_D,locate_dofs_topological(V, fdim, boundary_facets))

In [11]:
# build the stiffness matrix\

k_form = ufl.inner(epsilon(u),sigma(v))*dx
l_form = 1.*u[0]*dx # used to maintain symmetry
m_form = rho*ufl.inner(u,v)*dx

TypeError: create_matrix_nest(): incompatible function arguments. The following argument types are supported:
    1. (a: List[List[dolfinx::fem::Form<std::complex<double> >]], types: List[List[str]] = []) -> mat

Invoked with: [<dolfinx.cpp.fem.Form_complex128 object at 0x7f85d9863570>, <dolfinx.cpp.fem.Form_complex128 object at 0x7f85da79ca30>], []

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.

In [19]:
K_mat = dolfinx.fem.assemble_matrix(k_form)
K_mat.assemble()
B_mat = dolfinx.fem.assemble_vector(l_form)
B_mat.assemble()

In [12]:
problem = LinearProblem(k_form, l_form, bcs=[bc])
problem.A

In [13]:
K_mat = problem.A
K_mat.assemble()

In [14]:
# assemble the matricies
M_mat = assemble_matrix(m_form,bcs=[bc])
M_mat.assemble()

In [47]:
ki, kj, kv = K_mat.getValuesCSR()
Mi,Mj,Mv = M_mat.getValuesCSR() 

In [21]:
K_mat.getSize()

(50652, 50652)

In [15]:
# kmat_numpy = kmat_numpy.getDenseArray()
import scipy.linalg as splg
from scipy.sparse import linalg
import scipy.sparse as spsparse

In [50]:
Ksparse = spsparse.csr_matrix((kv,kj,ki))
Msparse = spsparse.csr_matrix((Mv,Mj,Mi))

ValueError: unable to infer matrix dimensions

In [29]:
# check if symmetric
print((np.abs(Ksparse - Ksparse.T)>1e-10).nnz == 0)
print((np.abs(Msparse - Msparse.T)>1e-10).nnz == 0)

False
True


In [195]:
%time w,v = spsparse.linalg.eigs(Ksparse,100,Msparse,which='SM')

CPU times: user 96.1 ms, sys: 20 ms, total: 116 ms
Wall time: 113 ms


In [196]:
w

array([9.64794903e+09+5.39260048e-05j, 9.22454365e+09+1.54667551e-06j,
       9.23273949e+09+2.80488433e-06j, 7.43933616e+09+9.29326496e-06j,
       7.06741142e+09+3.70421187e-05j, 6.64973353e+09+1.00591951e-05j,
       5.92241812e+09+4.06845157e-05j, 5.71601903e+09-7.60409241e-06j,
       5.29776537e+09-5.37452298e-06j, 5.20957243e+09+8.42202168e-05j,
       5.16927829e+09-1.61314127e-05j, 4.74020935e+09-2.64226429e-05j,
       4.61523647e+09+3.86903909e-05j, 4.30629492e+09+9.35631597e-05j,
       4.20577292e+09-3.16502876e-05j, 9.70333058e+07-8.93788220e-06j,
       3.69719241e+09+6.49279740e-05j, 3.70195814e+09-6.23596735e-05j,
       3.70329759e+09-2.07625189e-05j, 3.12304512e+08+2.77288300e-05j,
       4.91778721e+08+1.83789352e-06j, 4.27685458e+08-1.18731780e-05j,
       4.26059863e+08+1.15103660e-05j, 6.90821093e+08+3.33864881e-05j,
       7.02940475e+08+1.86884199e-06j, 3.23676549e+09+3.99328254e-06j,
       3.06798456e+09+3.85064169e-05j, 3.03455862e+09+1.39664906e-05j,
      

In [20]:
# set up eigensolver parameters
eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
eigensolver.setOperators(K_mat,M_mat)
# eigensolver.setDimensions(nev=10) # returns 10 values
# eigensolver.setProblemType(2) # Generalized Hermitian Eigensolver Problem
eigensolver.setWhichEigenpairs(2) # smallest eigenvalues
eigensolver.setFromOptions() # applies options
eigensolver.view() # checks results


EPS Object: 1 MPI processes
  type: krylovschur
    0% of basis vectors kept after restart
    using the locking variant
  problem type: not yet set
  selected portion of the spectrum: smallest eigenvalues in magnitude
  postprocessing eigenvectors with purification
  number of eigenvalues (nev): 1
  number of column vectors (ncv): -2
  maximum dimension of projected problem (mpd): -2
  maximum number of iterations: -2
  tolerance: -2.
  convergence test: relative to the eigenvalue
BV Object: 1 MPI processes
  type: svec
  0 columns of global length -1
  vector orthogonalization method: classical Gram-Schmidt
  orthogonalization refinement: if needed (eta: 0.7071)
  block orthogonalization method: GS
  doing matmult as a single matrix-matrix product
DS Object: 1 MPI processes
  type: nhep
ST Object: 1 MPI processes
  type: shift
  shift: 0.
  number of matrices: 2
  all matrices have unknown nonzero pattern
  KSP Object: (st_) 1 MPI processes
    type: preonly
    maximum iterations=10

In [ ]:
%time eigensolver.solve()

In [18]:
print(f'N_iterations: {eigensolver.getIterationNumber()}')
# k = eigensolver.getEigenpair(0)
# print(k)
eigensolver.getConverged()


N_iterations: 1


1

In [44]:
for eig_idx in range(eigensolver.getConverged()):
    k_sub = eigensolver.getEigenpair(eig_idx)
    print(f'Eig Freq {eig_idx}: {np.sqrt(np.abs(k_sub))/(2*np.pi)} Hz')

Eig Freq 0: 0.0 Hz


# Compute Theoretical Models from Leissa

Using Leissa to compute the models for a free-free plate

In [133]:
# leissa modes
import numpy as np

def G(m):
    return m-1/2

def J(m):
    return (m-1/2)**2*(1-(2/((m-1/2)*np.pi)))

def H(m):
    return (m-1/2)**2*(1-(2/((m-1/2)*np.pi)))

D = E*0.01**3/(12*(1-nu**2))
a = 1.0
b = 1.0
m = 2
n = 2
np.pi**4*D/(a**4*rho)*((G(m)**4+G(n)**4)*(a/b)**4+2*(a/b)**2*(nu*H(m)*H(n)+(1-nu)*J(m)*J(n)))

3183.4431698477947

In [124]:
B = 10.4
B*np.sqrt((E*0.01**2/(rho*a**4*(1-nu**2))))

560.9669245945568